# Estrategia de inversión a largo plazo (SPY)
- Alejandra Páez.
- Givanna Romero
- Oscar Troncoso.

<center><img src= "https://previews.123rf.com/images/maxkabakov/maxkabakov1703/maxkabakov170300344/73196110-stock-market-indexes-concept-pixelated-black-text-s-p-500-on-digital-background.jpg" title = 'Logo SPY' width="500px" ></center>

- Se van a tomar desde el primero de Enero del 2020 hasta la actualidad dado el crecimiento de de la compañia durante la pandemia del COVID - 19, sin embargo al momento de realizar el backtesting se van a tomar todos los datos históricos con el objetivo de entrenar el modelo.
- Elegimos un ETF del índice Standard & Poor’s 500 el cual se basa en las 500 empresas más grandes
de Estados Unidos donde debido a su rendimiento se puede ver reflejado como se encuentra el
bienestar del mercado. Este ETF es el mayor cotizado en EE.UU, el mejor reconocido y el más antiguo
ya que gracias al su gran volumen de negociación puede seguir muy de cerca al índice.
En este trabajo decidimos enfocarnos en una estrategia a largo plazo. Planteamos 3 estrategias con
diferentes Indicadores para generar un análisis propio y observar cual resulta más efectiva a la hora
de seguir el comportamiento del SPY.

## Llamado de librerias

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import yfinance as yf 
import mplfinance as mpf
import ipywidgets as widgets
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
import pandas_ta as tap
import talib as ta
from mplfinance.original_flavor import candlestick_ohlc
from backtesting import Strategy
from backtesting import Backtest

## Primera estrategia

Se presenta una gráfica interactiva en donde se pueden modificar la temporalidad de las medias móviles de rápida, media y lenta reacción junto con los soportes y resistencias generados mediante algorítmos para el análisis técnico

$$
SMA_{(20, 50, 200) = i} = \dfrac{\sum^{T}_{t = i}{X_{t - i + 1 }}}{T}
$$

- Decidimos usar este indicador que emplea tres medias móviles simples: Una de rápida
reacción (20 periodos) para determinar el impulso y las zonas de soporte y resistencia; una
de media (50 periodos) y una de lenta reacción (200 periodos) para evaluar los objetivos de
corrección y retroceso.

- Lo que nos dice el indicador es que cuando la SMA de más rápida reacción cruza por encima
de la más lenta, daría una señal de compra y esta finalmente se confirma cuando la de 20 y
la SMA de 50 periodos están por encima de la media móvil simple de 200 periodos. Por el
contrario, para una señal de venta, la media móvil simple más rápida cruza por debajo de la
más lenta y se confirma cuando ambas están por debajo de la de 200 periodos.


In [2]:
def Analisys(Asset, S_date, E_date, MA1, MA2, MA3):                                 #Formato yy/mm/dd
    # Obtención de los datos de los activos en la lista
    Data = yf.download(tickers = Asset, start = S_date, end = E_date, group_by = 'ticker')
    
    # Funciones de soportes y resistencias
    def Support(Data, i):
        support = Data['Low'][i] < Data['Low'][i - 1] and Data['Low'][i] < Data['Low'][i + 1] and Data['Low'][i + 1] < Data['Low'][i + 2] and Data['Low'][i - 1] < Data['Low'][i - 2]
        return support 

    def Resistance(Data, i):
        resistance = Data['High'][i] > Data['High'][i - 1] and Data['High'][i] > Data['High'][i + 1] and Data['High'][i + 1] > Data['High'][i + 2] and Data['High'][i - 1] > Data['High'][i - 2]
        return resistance 

    S = np.mean(Data['High'] - Data['Low'])
    def Far_from_level(j):
        return np.sum([abs(j - X) < S for X in Levels]) == 0

    Levels = []
    Colors = []
    N = Data.shape[0]

    for i in range(2, (N - 2)):
        if Support(Data, i):
            j = Data['Low'][i]
            # print('---> Support: {}' .format(j.round(2)))
            # print('Far from level: ', Far_from_level(j))
            if Far_from_level(j):
                Levels.append((i, j))
                Colors.append('green')
        elif Resistance(Data, i):
            j = Data['High'][i]
            # print('---> Support: {}' .format(j.round(2)))
            # print('Far from level: ', Far_from_level(j))
            if Far_from_level(j):
                Levels.append((i, j))
                Colors.append('red')
    
    # Medias moviles simples (SMA)
    MA_1 = Data['Close'].rolling(MA1).mean()
    MA_2 = Data['Close'].rolling(MA2).mean()
    MA_3 = Data['Close'].rolling(MA3).mean()
    
    Data = Data.reset_index()
    
    # Gráficas
    def Plot_all():
        ohlc = Data.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
        ohlc['Date'] = pd.to_datetime(ohlc['Date'])
        ohlc['Date'] = ohlc['Date'].apply(mpl_dates.date2num)
        ohlc = ohlc.astype(float)
        
        # Creating Subplots
        fig, ax = plt.subplots(figsize = (18, 7.5))
        candlestick_ohlc(ax, ohlc.values, width = 0.6, colorup = 'green', colordown = 'red', alpha = 1)
        
        # Setting labels & titles
        ax.set_xlabel('Date (mm / dd / yy)')
        ax.set_ylabel('USD Price')
        fig.suptitle('{} Soportes, resistencias y medias móviles' .format(Asset))

        # X axis data
        X = ohlc['Date']
        
        # Setting the simple moving average (SMA)
        ax.plot(X, MA_1, color = 'yellow', label= 'SMA {}' .format(MA1) , alpha = 0.8)
        ax.plot(X, MA_2, color = 'orange', label= 'SMA {}' .format(MA2) , alpha = 0.8)
        ax.plot(X, MA_3, color = 'blue', label= 'SMA {}' .format(MA3) , alpha = 0.8)
        plt.legend()
        
        # Setting the supports and resistances
        for Level, Color in zip(Levels, Colors):
                    plt.hlines(Level[1], xmin = X[Level[0]], \
                    xmax = max(X), colors = Color, linestyles = 'dotted', linewidth = 2.5)

        # Formatting Date
        date_format = mpl_dates.DateFormatter('%m - %d - %Y')
        ax.xaxis.set_major_formatter(date_format)
        fig.autofmt_xdate()
        fig.tight_layout()
        plt.show()
    Plot_all()
    
        
# Creación de los sliders
Analisys_controls = widgets.interactive(Analisys,
                                    Asset = widgets.Text(value = 'SPY', placeholder = 'Type something', description = 'Asset name', disabled = False),
                                    S_date = widgets.DatePicker(description = 'Initial Date', disabled=False),
                                    E_date = widgets.DatePicker(description = 'End Date', disabled=False),
                                    MA1 = widgets.IntSlider(value = 20, min = 0, max = 200),
                                    MA2 = widgets.IntSlider(value = 50, min = 0, max = 200),
                                    MA3 = widgets.IntSlider(value = 200, min = 0, max = 200)
                                    
)

display(Analisys_controls)

interactive(children=(Text(value='SPY', description='Asset name', placeholder='Type something'), DatePicker(va…

## Segunda estrategia

- Se presenta un gráfico donde se utilizan las bandas de Bollinger con una media movil de 20 periodos con bandas a 1.5 y 2 desviaciones estandar de la media junto con los soportes y resistencias claves del modelo generadas por un algorítmo.

- Las BB es un indicador envolvente de media móvil que tiene en cuenta la volatilidad de los
movimientos de precios. Estas líneas o bandas están ubicadas a 1.5 y 2 desviaciones
estándar de la media móvil de 20 periodos. Su funcionamiento esta dado de la siguiente
forma: Cuanto más cerca estén las bandas entre sí, menor será la volatilidad percibida y
viceversa. Los cambios de tendencia se inician principalmente por la ruptura del rango
anterior, materializado por una vela fuera de una de las dos bandas externas, muy por
encima o por debajo de los movimientos recientes. En este caso, la banda superior puede
considerarse un punto de resistencia y la banda inferior un nivel de soporte, de modo que,
si el precio rompe por encima de la resistencia o por debajo del soporte de estas bandas, se
podría considerar una oportunidad de trading.
- Su uso es estratégico debido a que, comparar simplemente precios con la media móvil
puede proporcionar señales falsas. Por lo tanto, el uso de un indicador de volatilidad puede
ser una forma efectiva de mitigar este problema en cierta medida. 

**MACD: Moving average convergene divergence**
$$
\alpha =  \dfrac{2}{1 + Time_{periods}}
$$

$$
EMA_{0} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t}}}{T}
$$

$$
EMA_{9, 12, 16} = \left(\$Price_{t = 0} * \dfrac{\alpha}{1 + Days}\right) + EMA_{t = -1} * \left(1 - \dfrac{\alpha}{1 + Days}\right)
$$

$$
MACD_t = EMA_{12}(t) - EMA_{26}(p) \qquad
Signal_t = EMA_{9}{t}
$$

**Bollinger Bands:**
$$
BB_{Upper} = BB_{Midle} + (2.5 \sigma^2)
$$

$$
BB_{Midle} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t}}}{T}
$$

$$
BB_{Lower} = BB_{Midle} + (2.5 \sigma^2)
$$

In [3]:
def Analisys(Asset, S_date, E_date):                                 #Formato yy/mm/dd
    # Obtención de los datos de los activos en la lista
    Data = yf.download(tickers = Asset, start = S_date, end = E_date, group_by = 'ticker')    
    
    # trim volume to avoid exponential form
    Data['Volume'] = Data['Volume'] / 1000

    # Macd
    Data["Macd"], Data["Macd_signal"], Data["Macd_hist"] = ta.MACD(Data['Close'])
    
    # Bolligner Bands
    Upperband, Midleband, Lowerband = ta.BBANDS(Data['Close'], 20, 1.5, 2)

    # Macd panel
    colors = ['g' if v >= 0 else 'r' for v in Data["Macd_hist"]]
    Macd_plot = mpf.make_addplot(Data["Macd"], panel = 2, color = 'blue', ylabel = "MACD", secondary_y=False)
    Macd_hist_plot = mpf.make_addplot(Data["Macd_hist"], type = 'bar', panel = 2, color = colors) # color='dimgray'
    Macd_signal_plot = mpf.make_addplot(Data["Macd_signal"], panel = 2, color = 'orange', secondary_y=False)

    # Bollinger panel
    Upperband_B = mpf.make_addplot(Upperband, panel = 0, color = 'blue')
    Midleband_B = mpf.make_addplot(Midleband, panel = 0, color = 'orange')
    Lowerband_B = mpf.make_addplot(Lowerband, panel = 0, color = 'blue')

    # Plot
    Plots = [Macd_plot, Macd_signal_plot, Macd_hist_plot, Upperband_B, Midleband_B, Lowerband_B]
    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)

# Creación de los sliders
Analisys_controls = widgets.interactive(Analisys,
                                    Asset = widgets.Text(value = 'SPY', placeholder = 'Type something', description = 'Asset name', disabled = False),
                                    S_date = widgets.DatePicker(description = 'Initial Date', disabled=False),
                                    E_date = widgets.DatePicker(description = 'End Date', disabled=False)
)

display(Analisys_controls)

interactive(children=(Text(value='SPY', description='Asset name', placeholder='Type something'), DatePicker(va…

## Estrategia Final

- Elegimos la estrategia 3, la cual está compuesta por el MACD, las bandas de bollinger,
triple media móvil exponencial y el índice de fuerza relativa. En donde el MACD nos
arroja la divergencia o convergencia que se encuentra en el activo durante un determinado
tiempo con dos medias móviles del precio las cuales son 12 y 26 y una señal de 9 , para
nuestro caso usamos utilizamos un EMA de 9, 12 y 26. En cuanto las bandas de bollinger
estas nos ayudan a identificar el rango en el cual opera el activo donde la línea superior es
la media más el precio, la línea del medio es una media móvil de 20 periodos y la línea
inferior es la media menos el precio del activo. La triple media exponencial nos ayuda a
suavizar los movimientos del mercado con el fin de obtener una línea continua de la
tendencia con el fin de que los otros osciladores nos ayuden a determinar qué sucederá
después; en el activo utilizamos un EMA de 10, 20 y 50, y por último el RSI nos brinda
información de la relación de los movimientos bullish y bearish dándonos valores entre de
0 a 100; para este activo lo estamos evaluando entre 70 y 30 ya que no es un activo tan
volátil. 

**MACD: Moving average convergene divergence**
$$
\alpha =  \dfrac{2}{1 + Time_{periods}}
$$

$$
EMA_{0} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t}}}{T}
$$

$$
EMA_{9, 12, 16} = \left(\$Price_{t = 0} * \dfrac{\alpha}{1 + Days}\right) + EMA_{t = -1} * \left(1 - \dfrac{\alpha}{1 + Days}\right)
$$

$$
MACD_t = EMA_{12}(t) - EMA_{26}(p) \qquad
Signal_t = EMA_{9}{t}
$$


**Bollinger Bands:**
$$
BB_{Upper} = BB_{Midle} + (2.5 \sigma^2)
$$

$$
BB_{Midle} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t}}}{T}
$$

$$
BB_{Lower} = BB_{Midle} + (2.5 \sigma^2)
$$

**TEMA: Triple exponential moving average** 
$$
\alpha =  \dfrac{2}{1 + Time_{periods}}
$$
$$
EMA_{0} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t}}}{T}
$$
$$
EMA_{10, 20, 50} = \left(\$Price_{t = 0} * \dfrac{\alpha}{1 + Days}\right) + EMA_{t = -1} * \left(1 - \dfrac{\alpha}{1 + Days}\right)
$$

**RSI: Relative stregth index**
$$
Price_{Up} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t > Close_{t-1}}}}{T_{Close_t > Close_{t-1}}} \qquad
Price_{Down} = \dfrac{\sum^{T}_{t = 1}{\$Price_{Close_t < Close_{t-1}}}}{T_{Close_t < Close_{t-1}}}
$$

$$
RSI = 100 - \dfrac{100}{1 - \dfrac{Price_{Up}}{Price_{Down}}}
$$

In [4]:
def Analisys(Asset, S_date, E_date, MACD = False, BollingerB = False, EMA = False, RSI = False):                                 #Formato yy/mm/dd
    # Obtención de los datos de los activos en la lista
    Data = yf.download(tickers = Asset, start = S_date, end = E_date, group_by = 'ticker')    
    
    # trim volume to avoid exponential form
    Data['Volume'] = Data['Volume'] / 1000

    # Macd
    Data["Macd"], Data["Macd_signal"], Data["Macd_hist"] = ta.MACD(Data['Close'])
    
    # Bolligner Bands
    Upperband, Midleband, Lowerband = ta.BBANDS(Data['Close'], 20, 1.5, 2)
    
    # EMA 5, 10, 20
    EMA10 = ta.EMA(Data['Close'], timeperiod = 10)
    EMA20 = ta.EMA(Data['Close'], timeperiod = 20)
    EMA50 = ta.EMA(Data['Close'], timeperiod = 50)
    
    # RSI
    RSI_C = ta.RSI(Data['Close'], timeperiod = 14)
    RSI_EMA = ta.EMA(RSI_C, timeperiod = 100)

    # Macd panel
    colors = ['g' if v >= 0 else 'r' for v in Data["Macd_hist"]]
    Macd_plot = mpf.make_addplot(Data["Macd"], panel = 2, color = 'blue', ylabel = "MACD", secondary_y=False)
    Macd_hist_plot = mpf.make_addplot(Data["Macd_hist"], type = 'bar', panel = 2, color = colors) # color='dimgray'
    Macd_signal_plot = mpf.make_addplot(Data["Macd_signal"], panel = 2, color = 'orange', secondary_y=False)

    # Bollinger panel
    Upperband_B = mpf.make_addplot(Upperband, panel = 0, color = 'blue')
    Midleband_B = mpf.make_addplot(Midleband, panel = 0, color = 'orange')
    Lowerband_B = mpf.make_addplot(Lowerband, panel = 0, color = 'blue')
    
    # EMA 5, 10, 20 panel
    EMA10_G =  mpf.make_addplot(EMA10, panel = 0, color = 'orange')
    EMA20_G =  mpf.make_addplot(EMA20, panel = 0, color = 'yellow')
    EMA50_G =  mpf.make_addplot(EMA50, panel = 0, color = 'blue')
    
    
    # Plot
    # Plots = [EMA10_G, EMA20_G, EMA50_G, Macd_plot, Macd_signal_plot, Macd_hist_plot, Upperband_B, Midleband_B, Lowerband_B, RSI_G]
    if MACD == False:                       # Ninguno 
        if BollingerB == False:
            if EMA == False:
                if RSI == False:
                    mpf.plot(Data, type = 'candle', style = 'yahoo', volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1), show_nontrading = True)
        
    if MACD == True:                        # Solo MACD
        if BollingerB == False:
            if EMA == False:
                if RSI == False:
                    Plots = [Macd_plot, Macd_signal_plot, Macd_hist_plot]
                    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 2),show_nontrading = True)       
              
    if BollingerB == True:                  # Solo Bollinger
        if MACD == False:
            if EMA == False:
                if RSI == False:
                    Plots = [Upperband_B, Midleband_B, Lowerband_B]
                    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1),show_nontrading = True)  
    
    if EMA == True:                         # Solo EMA
        if BollingerB == False:
            if MACD == False:
                if RSI == False:
                    Plots = [EMA10_G, EMA20_G, EMA50_G]
                    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1), show_nontrading = True)     
    
    if RSI == True:                         # Solo RSI
        if BollingerB == False:
            if MACD == False:
                if EMA == False:
                    RSI_G = mpf.make_addplot(RSI_C, panel = 2, color = 'purple', ylabel = 'RSI')
                    RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 2, color = 'yellow')
                    Plots = [RSI_G, RSI_EMA_G]
                    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)   
    
    if BollingerB and MACD == True:         # Bollinger y MACD
        if EMA == False:
            if RSI == False:
                Plots = [Macd_plot, Macd_signal_plot, Macd_hist_plot, Upperband_B, Midleband_B, Lowerband_B]
                mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)
            
    if EMA and BollingerB == True:          # EMA y Bollinger
        if MACD == False:
            if RSI == False:
                Plots = [EMA10_G, EMA20_G, EMA50_G, Upperband_B, Midleband_B, Lowerband_B]
                mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1), show_nontrading = True)
            
    if EMA and MACD == True:                # EMA y MACD
        if BollingerB == False:
            if RSI == False:
                Plots = [EMA10_G, EMA20_G, EMA50_G, Macd_plot, Macd_signal_plot, Macd_hist_plot]
                mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)

    if RSI and BollingerB == True:          # RSI y Bollinger
        if EMA == False:
            if MACD == False:
                RSI_G = mpf.make_addplot(RSI_C, panel = 2, color = 'purple', ylabel = 'RSI')
                RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 2, color = 'yellow')
                Plots = [RSI_G, Upperband_B, Midleband_B, Lowerband_B, RSI_EMA_G]
                mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True) 
            
    if RSI == True:                         # RSI y MACD
        if MACD == True:
            if EMA == False:
                if BollingerB == False:
                    RSI_G = mpf.make_addplot(RSI_C, panel = 3, color = 'purple', ylabel = 'RSI')
                    RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 3, color = 'yellow')
                    Plots = [Macd_plot, Macd_signal_plot, Macd_hist_plot, RSI_G, RSI_EMA_G]
                    mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2, 2), show_nontrading = True)           
                
    if RSI and EMA == True:                 # RSI y EMA
        if BollingerB == False:
            if MACD == False:
                RSI_G = mpf.make_addplot(RSI_C, panel = 2, color = 'purple', ylabel = 'RSI')
                RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 2, color = 'yellow')
                Plots = [EMA10_G, EMA20_G, EMA50_G, RSI_G, RSI_EMA_G]
                mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)  
 
    if EMA and BollingerB and MACD == True: # EMA, Bollinger y MACD
        if RSI == False: 
            Plots = [EMA10_G, EMA20_G, EMA50_G, Macd_plot, Macd_signal_plot, Macd_hist_plot, Upperband_B, Midleband_B, Lowerband_B]
            mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)
            
    if RSI and BollingerB and MACD == True: # RSI, Bollinger y MACD
        if EMA == False:
            RSI_G = mpf.make_addplot(RSI_C, panel = 3, color = 'purple', ylabel = 'RSI')
            RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 3, color = 'yellow') 
            Plots = [RSI_G, Macd_plot, Macd_signal_plot, Macd_hist_plot, Upperband_B, Midleband_B, Lowerband_B, RSI_EMA_G]
            mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2, 2), show_nontrading = True)
            
    if RSI and BollingerB and EMA == True:  # RSI, Bollinger y EMA
        if MACD == False:
            RSI_G = mpf.make_addplot(RSI_C, panel = 2, color = 'purple', ylabel = 'RSI')
            RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 2, color = 'yellow')  
            Plots = [RSI_G, EMA10_G, EMA20_G, EMA50_G, Upperband_B, Midleband_B, Lowerband_B, RSI_EMA_G]
            mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2), show_nontrading = True)
    
    if RSI and EMA and MACD == True:        # RSI, EMA y MACD
        if BollingerB == False:
            RSI_G = mpf.make_addplot(RSI_C, panel = 3, color = 'purple', ylabel = 'RSI')
            RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 3, color = 'yellow') 
            Plots = [RSI_G, Macd_plot, Macd_signal_plot, Macd_hist_plot, EMA10_G, EMA20_G, EMA50_G, RSI_EMA_G]
            mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2, 2), show_nontrading = True)
    
    if RSI and EMA and MACD and BollingerB == True: # RSI, EMA, MACD y Bollinger
        RSI_G = mpf.make_addplot(RSI_C, panel = 3, color = 'purple', ylabel = 'RSI') 
        RSI_EMA_G = mpf.make_addplot(RSI_EMA, panel = 3, color = 'yellow') 
        Plots = [RSI_G, Macd_plot, Macd_signal_plot, Macd_hist_plot, EMA10_G, EMA20_G, EMA50_G, Upperband_B, Midleband_B, Lowerband_B, RSI_EMA_G]
        mpf.plot(Data, type = 'candle', style = 'yahoo', addplot = Plots, volume = True, ylabel = 'Price USD', figscale = 1.8, panel_ratios = (4, 1, 2, 2), show_nontrading = True)
            
# Creación de los sliders
Analisys_controls = widgets.interactive(Analisys,
                                    Asset = widgets.Text(value = 'SPY', placeholder = 'Type something', description = 'Asset name', disabled = False),
                                    S_date = widgets.DatePicker(description = 'Initial Date', disabled = False),
                                    E_date = widgets.DatePicker(description = 'End Date', disabled = False),
                                    MACD = widgets.Checkbox(value = False, description = 'MACD (9, 12, 26)', disabled = False),
                                    BollingerB = widgets.Checkbox(value = False, description = 'Bollinger Bands (20, 1.5, 2)', disabled = False),
                                    EMA = widgets.Checkbox(value = False, description = 'TEMA (10, 20, 50)', disabled = False),
                                    RSI = widgets.Checkbox(value = False, description = 'RSI (14, 200)', disabled = False)
)

display(Analisys_controls)

interactive(children=(Text(value='SPY', description='Asset name', placeholder='Type something'), DatePicker(va…

## Backtesting de la estrategia
- Para el backtesting se usó las bandas de bollinger apoyadas con una media móvil ponderado de los
datos del RSI. Donde al final me arroja datos como la duración que me arroja los días, un tiempo de
exposición en porcentaje, la equidad final y su pico en dólares, el retorno, el porcentaje de la máxima
reducción , el promedio de la reducción en porcentaje, la cantidad de operaciones que se realizaron,
la tasa de ganancias, El porcentaje que se obtuvo en la mejor operación, en la peor y el promedio
de todas, la máxima duración días del comercio y su promedio, el factor de beneficio y por último
la expectativa en porcentaje que se tenía con la estrategia 

In [37]:
Asset = yf.download("SPY", start='2011-01-05', end='2021-01-05')

Asset = Asset[Asset.High!=Asset.Low]
Asset.reset_index(inplace=True)
Asset.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-05,126.580002,127.720001,126.459999,127.639999,102.045319,133975300
1,2011-01-06,127.690002,127.830002,127.010002,127.389999,101.845428,122519000
2,2011-01-07,127.559998,127.769997,126.150002,127.139999,101.645561,156034600
3,2011-01-10,126.580002,127.160004,126.199997,126.980003,101.517624,122401700
4,2011-01-11,127.440002,127.739998,126.949997,127.430000,101.877419,110287000


In [38]:
Asset['EMA'] = tap.sma(Asset.Close, length = 200)
Asset['RSI'] = tap.rsi(Asset.Close, length = 2)
My_bbands = tap.bbands(Asset.Close, length = 20, std = 2.5)
My_bbands[0:50]
Asset = Asset.join(My_bbands)
Asset.dropna(inplace=True)
Asset.reset_index(inplace=True)
Asset

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
0,199,2011-10-19,122.379997,123.080002,120.709999,121.129997,98.249275,226601300,127.676650,42.280175,108.019257,117.023000,126.026743,15.387989,0.728072
1,200,2011-10-20,121.430000,122.099998,119.820000,121.660004,98.679176,262075600,127.646750,55.526113,108.454065,117.463000,126.471935,15.339187,0.732936
2,201,2011-10-21,123.089996,124.120003,122.720001,123.970001,100.552803,278999400,127.629650,85.177375,108.609887,117.984500,127.359113,15.891262,0.819240
3,202,2011-10-24,124.169998,125.800003,124.059998,125.489998,101.785698,203215600,127.621400,92.104720,108.288293,118.447000,128.605708,17.153170,0.846648
4,203,2011-10-25,124.889999,124.949997,122.779999,123.050003,99.806602,268596800,127.601750,36.835441,108.277931,118.722500,129.167069,17.594927,0.707165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,2511,2020-12-28,371.739990,372.589996,371.070007,372.170013,363.070648,39000400,320.936701,93.927069,362.030797,368.011501,373.992204,3.250281,0.847661
2313,2512,2020-12-29,373.809998,374.000000,370.829987,371.459991,362.377991,53680500,321.594751,70.543834,363.281923,368.481500,373.681077,2.822165,0.786417
2314,2513,2020-12-30,372.339996,373.100006,371.570007,371.989990,362.894989,49455300,322.190701,78.525184,363.447835,368.780000,374.112165,2.891786,0.801002
2315,2514,2020-12-31,371.779999,374.660004,371.230011,373.880005,364.738831,78520700,322.860101,92.676970,363.257667,369.134500,375.011333,3.184115,0.903747


In [39]:
def Addemasignal(df, backcandles):
    emasignal = [0] * len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i] >= df.EMA[i]:
                dnt = 0
            if df.Low[i] <= df.EMA[i]:
                upt = 0
        if upt == 1 and dnt == 1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row] = 3
        elif upt == 1:
            emasignal[row] = 2
        elif dnt == 1:
            emasignal[row] = 1
    df['EMASignal'] = emasignal

Addemasignal(Asset, 6)

In [40]:
def Addorderslimit(df, percent):
    ordersignal = [0] * len(df)
    for i in range(1, len(df)):                                                     #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i] == 2 and df.Close[i] <= df['BBL_20_2.5'][i]:             # and df.RSI[i] <= 100: #Added RSI condition to avoid direct close condition
            ordersignal[i] = df.Close[i] - df.Close[i] * percent
        elif df.EMASignal[i] == 1 and df.Close[i] >= df['BBU_20_2.5'][i]:           # and df.RSI[i]>=0:
            ordersignal[i] = df.Close[i] + df.Close[i] * percent
    df['ordersignal'] = ordersignal
    
Addorderslimit(Asset, 0.00)

In [41]:
Asset[Asset.ordersignal != 0].head(10)

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal
118,317,2012-04-10,137.949997,138.339996,135.759995,135.899994,111.414528,235360300,127.20810,1.283267,136.965858,140.124999,143.284140,4.509032,-0.168695,2,135.899994
418,617,2013-06-20,161.860001,163.470001,158.979996,159.399994,133.586899,321255900,150.68315,9.640981,159.732008,163.917001,168.101993,5.106234,-0.039667,2,159.399994
420,619,2013-06-24,157.410004,158.429993,155.729996,157.059998,132.322372,222329000,150.85285,3.880312,157.280926,163.185501,169.090076,7.236641,-0.018708,2,157.059998
457,656,2013-08-15,167.410004,167.429993,166.089996,166.380005,140.174423,152931800,154.98480,4.873697,166.846459,169.221500,171.596540,2.807020,-0.098199,2,166.380005
458,657,2013-08-16,166.059998,166.630005,165.500000,165.830002,139.711044,130868200,155.10720,3.570811,166.044494,169.054500,172.064506,3.560989,-0.035630,2,165.830002
459,658,2013-08-19,165.639999,166.210007,164.759995,164.770004,138.817978,96437600,155.22430,1.758649,165.025234,168.818000,172.610766,4.493319,-0.033647,2,164.770004
568,767,2014-01-24,181.600006,181.660004,178.830002,178.889999,152.270874,208677100,170.21715,2.676726,180.195967,183.370500,186.545033,3.462425,-0.205694,2,178.889999
569,768,2014-01-27,179.059998,179.520004,177.119995,178.009995,151.521866,180843100,170.31125,1.964093,178.782915,183.078000,187.373084,4.692081,-0.089977,2,178.009995
622,821,2014-04-11,182.139999,183.419998,181.309998,181.509995,155.182907,167251000,176.22305,13.703061,181.599974,186.087500,190.575026,4.823028,-0.010026,2,181.509995
698,897,2014-07-31,195.610001,195.779999,192.970001,193.089996,165.874359,183479000,185.77720,1.029583,194.161743,197.180000,200.198256,3.061423,-0.177544,2,193.089996


In [42]:
def Pointposbreak(x):
    if x['ordersignal'] != 0:
        return x['ordersignal']
    else:
        return np.nan
Asset['pointposbreak'] = Asset.apply(lambda row: Pointposbreak(row), axis = 1)

In [43]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = Asset
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [44]:
dfpl = Asset[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [45]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            #print('!!!!!!!!!!!!!!!!!!!', self.data.index[-1])
            if self.data.index[-1]-self.ordertime[0]>5:#days max to fulfill the order!!!
                #print('----------------------')
                #print(self.orders)
                #print(self.ordertime)
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            #print(self.data.index[-1], self.trades)
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash = 10000, margin = 1/19, commission = 0.00)
stat = bt.run()
stat

C:\Users\oscar\AppData\Local\Temp\ipykernel_6436\3338304232.py:52: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2316.0
Duration                               2316.0
Exposure Time [%]                    4.013811
Equity Final [$]                343524.271164
Equity Peak [$]                 343524.271164
Return [%]                        3335.242712
Buy & Hold Return [%]              204.458034
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.663226
Avg. Drawdown [%]                   -18.06583
Max. Drawdown Duration                  327.0
Avg. Drawdown Duration                   60.5
# Trades                                 23.0
Win Rate [%]                        91.304348
Best Trade [%]                       2.181702
Worst Trade [%]                     -1.050928
Avg. Trade [%]                    

In [14]:
bt.plot(show_legend=False)

Row(id='1454', ...)

<center><img src= "https://i.ibb.co/KrHDLpg/image-2022-10-30-000250149.png" title = 'Logo SPY' width="1000px" ></center>

In [15]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1513,125,128,124.250000,122.519997,2617.495079,0.013924,125,128,3
1,2111,850,852,112.400002,112.269997,-274.440308,-0.001157,850,852,2
2,2076,930,935,111.809998,111.260002,-1141.790497,-0.004919,930,935,5
3,1660,1401,1406,126.849998,126.919998,116.199493,0.000552,1401,1406,5
4,1438,1700,1705,148.020004,147.279999,-1064.127899,-0.004999,1700,1705,5
5,-2122,2146,2149,90.879997,92.029999,-2440.303238,-0.012654,2146,2149,3
6,1138,2673,2676,129.039993,128.850006,-216.205414,-0.001472,2673,2676,3
7,896,3186,3190,159.399994,161.100006,1523.210938,0.010665,3186,3190,4
8,1033,3225,3230,166.059998,166.550003,506.175674,0.002951,3225,3230,5
9,1012,3336,3340,178.889999,177.009995,-1902.564941,-0.010509,3336,3340,4


**Conclusiones:**
Después de observar el comportamiento de la combinación de distintos indicadores en tres estrategias,
sé observo lo siguiente:
- Con las SMA Weighted de 20, 50 y 200 podemos ver que, si hay una tendencia definida, las SMA
de rápida y media reacción se cruzan constantemente.
- Con las BB y el MACD, se evidencia las bandas más cercanas lo que representa baja volatilidad
del ETF y el mercado ha mantenido en una tendencia (bearish) a partir de junio del 2022 con
algunos periodos de corrección alcista.
- Con el uso de estos 4 indicadores (MACD, BB, triple media móvil exponencial y RSI) se
confirma lo mencionado anteriormente: baja volatilidad, una tendencia bajista desde junio
del 2022 con algunos periodos de corrección alcista. Adicionalmente, un RSI donde el
mercado se ha mantenido en los niveles normales entre 30 y 70, por lo que utilizamos la de
70 porque es la resistencia que más respeta el activo y el soporte de 32.
A partir de lo anterior y con la realización del backtesting encontramos que la estrategia más eficaz
para operar este ETF es la tercera estrategia.

<center><img src= "https://pbs.twimg.com/profile_images/1250436908125061120/_OsvNasC_400x400.jpg" title = 'Logo SPY' width="500px" ></center>